In [1]:
# Installs
!conda install -c conda-forge geopy --yes #Geolocation library
!conda install -c conda-forge geocoder --yes #Geolocation library
!conda install -c conda-forge folium=0.5.0 --yes #Mapping library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [72]:
# Imports
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from zipfile import ZipFile # library for handling ZIP files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from requests import Request

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import folium # map rendering library
from folium import plugins

print('Libraries imported.')

Libraries imported.


### Download Dutch demographic data based on postal code

Dutch postal codes have 6 positions: four digits followed by two letters. The four digits are the best Dutch proxy of neighbourhoods.

The Dutch Central Bureau of Statistics has publised an Excel dataset of demographics per four digit postal code.

See https://www.cbs.nl/nl-nl/maatwerk/2016/51/bevolking-per-viercijferige-postcode-op-1-januari-2016

In [3]:
#Table of Dutch inhabitants per four digit postal code. Inhabitants split per age and sex, migration background and household composition.
#Excel has multi row header, so header is set to none. Columns BU and BV not imported as we do not need it
demo = pd.read_excel('https://www.cbs.nl/-/media/_excel/2016/51/bevolking-per-postcode-1-januari-2016.xls',
                     sheet_name = 'Bevolking per postcode', header = None, skiprows = 5, nrows = 4037, dtype = float, usecols = range(0, 72))
demo.columns = ['postcode', 'tot_inwoners',
                'tot_0-5', 'tot_5-10', 'tot_10-15', 'tot_15-20', 'tot_20-25', 'tot_25-30', 'tot_30-35', 'tot_35-40', 'tot_40-45', 'tot_45-50',
                'tot_50-55', 'tot_55-60', 'tot_60-65', 'tot_65-70', 'tot_70-75', 'tot_75-80', 'tot_80-85', 'tot_85-90', 'tot_90-95', 'tot_95-',
                'tot_man',
                'man_0-5', 'man_5-10', 'man_10-15', 'man_15-20', 'man_20-25', 'man_25-30', 'man_30-35', 'man_35-40', 'man_40-45', 'man_45-50',
                'man_50-55', 'man_55-60', 'man_60-65', 'man_65-70', 'man_70-75', 'man_75-80', 'man_80-85', 'man_85-90', 'man_90-95', 'man_95-',
                'tot_vrouw',
                'vrouw_0-5', 'vrouw_5-10', 'vrouw_10-15', 'vrouw_15-20', 'vrouw_20-25', 'vrouw_25-30', 'vrouw_30-35', 'vrouw_35-40', 'vrouw_40-45', 'vrouw_45-50',
                'vrouw_50-55', 'vrouw_55-60', 'vrouw_60-65', 'vrouw_65-70', 'vrouw_70-75', 'vrouw_75-80', 'vrouw_80-85', 'vrouw_85-90', 'vrouw_90-95', 'vrouw_95-',
                'tot_allochtoon', 'allochtoon_westers', 'allochtoon_nietwesters',
                'tot_huishoudens', 'eenspersoons', 'meer_zonder_kinderen', 'meer_met_kinderen',
                'gem_huish_grootte'               ]
demo['postcode'] = demo['postcode'].astype(int)
demo.head()

,postcode,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,9605.0,295.0,290.0,220.0,315.0,885.0,1100.0,835.0,650.0,...,25.0,5.0,3770.0,2490.0,1280.0,6325.0,4105.0,1360.0,860.0,1.51
1,1012,8245.0,220.0,135.0,115.0,160.0,1100.0,1705.0,1165.0,700.0,...,5.0,0.0,3795.0,2530.0,1270.0,5975.0,4265.0,1275.0,440.0,1.38
2,1013,21185.0,1110.0,855.0,675.0,690.0,1290.0,2075.0,2145.0,1910.0,...,40.0,10.0,9285.0,4300.0,4985.0,12640.0,7485.0,2630.0,2520.0,1.66
3,1014,580.0,30.0,15.0,5.0,15.0,25.0,135.0,125.0,60.0,...,0.0,0.0,225.0,135.0,85.0,295.0,135.0,115.0,45.0,1.74
4,1015,14860.0,575.0,465.0,430.0,445.0,1085.0,1605.0,1445.0,1105.0,...,20.0,5.0,5395.0,3695.0,1700.0,9850.0,6500.0,1955.0,1390.0,1.51


In [4]:
demo.shape

(4037, 72)

In [5]:
demo.dtypes

postcode                    int64
tot_inwoners              float64
tot_0-5                   float64
tot_5-10                  float64
tot_10-15                 float64
tot_15-20                 float64
tot_20-25                 float64
tot_25-30                 float64
tot_30-35                 float64
tot_35-40                 float64
tot_40-45                 float64
tot_45-50                 float64
tot_50-55                 float64
tot_55-60                 float64
tot_60-65                 float64
tot_65-70                 float64
tot_70-75                 float64
tot_75-80                 float64
tot_80-85                 float64
tot_85-90                 float64
tot_90-95                 float64
tot_95-                   float64
tot_man                   float64
man_0-5                   float64
man_5-10                  float64
man_10-15                 float64
man_15-20                 float64
man_20-25                 float64
man_25-30                 float64
man_30-35     

In [6]:
demo.describe()

,postcode,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
count,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,...,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000
mean,5675.431756,4205.483032,217.184791,230.355462,245.973495,252.276443,264.036413,265.252663,252.012633,250.016101,...,17.548922,4.478573,929.333664,410.021055,519.244488,1912.280159,719.728759,553.768888,638.679713,2.312747
std,2671.752148,4198.150829,243.444008,252.998869,259.059261,257.089459,359.385678,347.647781,307.972293,283.131502,...,24.795854,7.524776,1583.838690,543.553199,1166.762322,2012.592483,982.593115,532.601287,660.946912,0.326196
min,1011.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3332.000000,670.000000,30.000000,35.000000,40.000000,45.000000,35.000000,30.000000,30.000000,30.000000,...,0.000000,0.000000,45.000000,30.000000,10.000000,275.000000,70.000000,95.000000,105.000000,2.150000
50%,5711.000000,2665.000000,125.000000,135.000000,155.000000,165.000000,130.000000,125.000000,125.000000,140.000000,...,5.000000,0.000000,260.000000,175.000000,75.000000,1120.000000,315.000000,370.000000,395.000000,2.330000
75%,8051.000000,6885.000000,345.000000,365.000000,380.000000,395.000000,370.000000,390.000000,395.000000,400.000000,...,25.000000,5.000000,1185.000000,625.000000,510.000000,3125.000000,1050.000000,915.000000,1025.000000,2.500000
max,9999.000000,28315.000000,2125.000000,2320.000000,2395.000000,1855.000000,3665.000000,3130.000000,2720.000000,2155.000000,...,215.000000,70.000000,19785.000000,4380.000000,17385.000000,13200.000000,7995.000000,2635.000000,5595.000000,4.170000


In [7]:
#Convert all demographic data on number of inhabitants to fraction of total inhabitants in postal code
demo.iloc[:,2:67] = demo.iloc[:,2:67].div(demo.tot_inwoners, axis=0)
demo.head()

,postcode,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,9605.0,0.030713,0.030193,0.022905,0.032795,0.092140,0.114524,0.086934,0.067673,...,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,4105.0,1360.0,860.0,1.51
1,1012,8245.0,0.026683,0.016374,0.013948,0.019406,0.133414,0.206792,0.141298,0.084900,...,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,4265.0,1275.0,440.0,1.38
2,1013,21185.0,0.052396,0.040359,0.031862,0.032570,0.060892,0.097947,0.101251,0.090158,...,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,7485.0,2630.0,2520.0,1.66
3,1014,580.0,0.051724,0.025862,0.008621,0.025862,0.043103,0.232759,0.215517,0.103448,...,0.000000,0.000000,0.387931,0.232759,0.146552,295.0,135.0,115.0,45.0,1.74
4,1015,14860.0,0.038694,0.031292,0.028937,0.029946,0.073015,0.108008,0.097241,0.074361,...,0.001346,0.000336,0.363055,0.248654,0.114401,9850.0,6500.0,1955.0,1390.0,1.51


In [8]:
#Convert all demographic data on number of households to fraction of total households in postal code
demo.iloc[:,68:71] = demo.iloc[:,68:71].div(demo.tot_huishoudens, axis=0)
demo.head()

,postcode,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,9605.0,0.030713,0.030193,0.022905,0.032795,0.092140,0.114524,0.086934,0.067673,...,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51
1,1012,8245.0,0.026683,0.016374,0.013948,0.019406,0.133414,0.206792,0.141298,0.084900,...,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38
2,1013,21185.0,0.052396,0.040359,0.031862,0.032570,0.060892,0.097947,0.101251,0.090158,...,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66
3,1014,580.0,0.051724,0.025862,0.008621,0.025862,0.043103,0.232759,0.215517,0.103448,...,0.000000,0.000000,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74
4,1015,14860.0,0.038694,0.031292,0.028937,0.029946,0.073015,0.108008,0.097241,0.074361,...,0.001346,0.000336,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51


### Download geolocation data

This website offers datasets for geolocation per Dutch four digit postal codes.
https://git.tuxm.nl/tuxmachine/postcodes/src/4329c858db24b79523fd3fbbaf2df138ccaf16cd

Credit: https://git.tuxm.nl/tuxmachine/postcodes/src/master/README.md

License: https://git.tuxm.nl/tuxmachine/postcodes/src/master/LICENSE

In [9]:
#Four digit postal code LatLong in CSV dataset for Netherlands areas in LatLong geolocation
neighlatlong = pd.read_csv('https://git.tuxm.nl/tuxmachine/postcodes/raw/master/4pp.csv')
neighlatlong.head()

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,1,1000,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.336243,4.869444,Postbus
1,2,1001,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Postbus
2,3,1002,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Onbekend
3,4,1003,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Onbekend
4,5,1005,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Postbus


In [10]:
neighlatlong.shape

(4699, 10)

Drop all areas with category ('Soort') Postbus (=P.O. Box) or Onbekend (=Unkown)

In [11]:
#Drop all rows with Soort is Onbekend
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Onbekend'].index, inplace=True)
#Drop all rows with Soort is Postbus
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Postbus'].index, inplace=True)
neighlatlong.head()

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres


Drop all cities not being the cities we want to research: Amsterdam, Rotterdam, The Hague and Utrecht

In [12]:
# List of cities to select
cityselect = ['amsterdam','rotterdam','den haag','utrecht']
neighlatlong = neighlatlong[neighlatlong.woonplaats.str.contains('|'.join(cityselect), case=False)]
neighlatlong

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres
14,15,1016,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.371147,4.883705,Adres
15,16,1017,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364185,4.890775,Adres
16,17,1018,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.368387,4.918091,Adres
17,18,1019,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372858,4.938828,Adres
19,20,1021,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.384278,4.920920,Adres


In [13]:
#Oops, we have a city called Nieuw-Amsterdam completely somewhere else in the Netherlands. Drop that.
neighlatlong.drop(neighlatlong[neighlatlong.woonplaats == 'Nieuw-Amsterdam'].index, inplace=True)
neighlatlong

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres
14,15,1016,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.371147,4.883705,Adres
15,16,1017,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364185,4.890775,Adres
16,17,1018,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.368387,4.918091,Adres
17,18,1019,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372858,4.938828,Adres
19,20,1021,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.384278,4.920920,Adres


In [14]:
neighlatlong.shape

(252, 10)

In [15]:
neighlatlong.dtypes

id                              int64
postcode                        int64
woonplaats                     object
alternatieve_schrijfwijzen     object
gemeente                       object
provincie                      object
netnummer                       int64
latitude                      float64
longitude                     float64
soort                          object
dtype: object

In [16]:
neighlatlong.describe()

,id,postcode,netnummer,latitude,longitude
count,252.000000,252.000000,252.000000,252.000000,252.000000
mean,715.353175,2367.206349,31.269841,52.131220,4.681036
std,491.045853,964.835282,22.879700,0.176728,0.306865
min,10.000000,1011.000000,10.000000,51.871742,4.218229
25%,77.750000,1086.750000,10.000000,51.965608,4.421880
50%,817.500000,2571.500000,20.000000,52.085449,4.665768
75%,1062.250000,3068.250000,30.000000,52.344597,4.937426
max,1356.000000,3585.000000,70.000000,52.423834,5.164979


### CBS income data per postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on standardised income based on postal code.
See https://www.cbs.nl/nl-nl/maatwerk/2017/15/besteedbaar-inkomen-per-postcodegebied-2004-2014

In [17]:
#Excel has multi row header, so header is set to none. Only columns for latest year (2014) are imported.
#Columns are: postcode for first four digits postal code, gem_inkomen is average income, std_inkomen is standardised income corrected for householdsize
inkomen = pd.read_excel('https://www.cbs.nl/-/media/_excel/2017/15/besteedbaar-inkomen-postcode-2004-2014.xlsx',
                        sheet_name = 'Tabel1', header = None, usecols = [0, 23, 35], skiprows = 10, skipfooter = 5)
inkomen.columns = ['postcode', 'gem_inkomen', 'std_inkomen' ]
inkomen.head()

,postcode,gem_inkomen,std_inkomen
0,1011,32,26.3
1,1012,26.2,22.4
2,1013,30.3,24.1
3,1014,35,28.5
4,1015,33.3,27.2


In [18]:
#Check data types
inkomen.dtypes

postcode        int64
gem_inkomen    object
std_inkomen    object
dtype: object

In [19]:
#Two income columns have converted to Object instead of Float due to fields containing - instead of zero. Correct this
inkomen['gem_inkomen'] = pd.to_numeric(inkomen['gem_inkomen'], errors='coerce')
inkomen['gem_inkomen'].fillna(0, inplace=True)
inkomen['std_inkomen'] = pd.to_numeric(inkomen['std_inkomen'], errors='coerce')
inkomen['std_inkomen'].fillna(0, inplace=True)
inkomen.head()

,postcode,gem_inkomen,std_inkomen
0,1011,32.0,26.3
1,1012,26.2,22.4
2,1013,30.3,24.1
3,1014,35.0,28.5
4,1015,33.3,27.2


In [20]:
inkomen.shape

(4053, 3)

In [21]:
inkomen.dtypes

postcode         int64
gem_inkomen    float64
std_inkomen    float64
dtype: object

In [22]:
inkomen.describe()

,postcode,gem_inkomen,std_inkomen
count,4053.000000,4053.000000,4053.000000
mean,5673.362941,32.684160,22.438959
std,2671.635711,14.333122,9.528407
min,1011.000000,0.000000,0.000000
25%,3331.000000,30.000000,21.800000
50%,5709.000000,36.400000,25.000000
75%,8045.000000,40.700000,27.200000
max,9999.000000,108.800000,76.800000


### CBS names of areas and neighbourhoods based on postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on areas and neighbourhoods based on postal code.

See: https://www.cbs.nl/nl-nl/maatwerk/2018/36/buurt-wijk-en-gemeente-2018-voor-postcode-huisnummer

In [23]:
#Download and extract ZIP file of CBS area and neighbourhood data
!wget -q -O '2018-cbs-pc6huisnr20180801_buurt-vs2.zip' https://www.cbs.nl/-/media/_excel/2018/36/2018-cbs-pc6huisnr20180801_buurt%20-vs2.zip
with ZipFile('2018-cbs-pc6huisnr20180801_buurt-vs2.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
print('Data downloaded!')

Data downloaded!


In [24]:
#Create dataframe on lowlevel postal code data
pc = pd.read_csv('pc6hnr20180801_gwb-vs2.csv', sep=';', encoding='latin_1')
pc.rename(columns = {'Buurt2018': 'buurtcode', 'Wijk2018': 'wijkcode', 'Gemeente2018': 'gemeentecode'}, inplace = True)
pc.head()

,PC6,Huisnummer,buurtcode,wijkcode,gemeentecode
0,1011AB,105,3630400,36304,363
1,1011AB,106,3630400,36304,363
2,1011AB,107,3630400,36304,363
3,1011AB,110,3630400,36304,363
4,1011AB,112,3630400,36304,363


In [25]:
#Create dataframe with neighbourhood codes and names
wijk = pd.read_csv('wijknaam2018.csv', sep=';', encoding='latin_1')
wijk.rename(columns = {'GWBcode8': 'wijkcode', 'GWBlabel': 'wijk'}, inplace = True)
wijk.head()

,wijkcode,wijk
0,300,Wijk 00
1,500,Wijk 00
2,900,Wijk 00 West
3,901,Wijk 01 Oost
4,1000,Wijk 00 Stad


In [26]:
#Create dataframe with area codes and names
buurt = pd.read_csv('buurtnaam2018.csv', sep=';', encoding='latin_1')
buurt.rename(columns = {'GWBcode8': 'buurtcode', 'GWBlabel': 'buurt'}, inplace = True)
buurt.head()

,buurtcode,buurt
0,30000,Appingedam-Centrum
1,30001,Appingedam-West
2,30002,Appingedam-Oost
3,30007,Verspreide huizen Damsterdiep en Eemskanaal
4,30008,Verspreide huizen ten zuiden van Eemskanaal


In [27]:
#Merge postal code dataframe with neighbourhood and area names
pcmerge = pd.merge(pc, wijk, on='wijkcode')
pcmerge = pd.merge(pcmerge, buurt, on='buurtcode')
pcmerge.head()

,PC6,Huisnummer,buurtcode,wijkcode,gemeentecode,wijk,buurt
0,1011AB,105,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
1,1011AB,106,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
2,1011AB,107,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
3,1011AB,110,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
4,1011AB,112,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland


In [28]:
#Drop columns we will not need: housenumber (too low level), Municipality (will be added by geocode data by name instead of code) and the codes
pcmerge.drop(['Huisnummer', 'buurtcode', 'wijkcode', 'gemeentecode'], axis=1, inplace = True)
pcmerge.head()

,PC6,wijk,buurt
0,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
1,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
2,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
3,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
4,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland


In [29]:
#Create column with first four digits of postal code as all six is too low level
pcmerge['postcode'] = pcmerge['PC6'].str[:4]
pcmerge.head()

,PC6,wijk,buurt,postcode
0,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
1,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
2,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
3,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
4,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011


In [30]:
#Drop PC6 and group on postcode
postcode = pcmerge.groupby(['postcode'], as_index=False).first()
postcode.drop(['PC6'], axis=1, inplace = True)
postcode.head()

,postcode,wijk,buurt
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.
2,1013,Haarlemmerbuurt,Westerdokseiland
3,1014,Westelijk Havengebied,Alfa-driehoek
4,1015,Grachtengordel-West,Langestraat e.o.


In [31]:
postcode.dtypes

postcode    object
wijk        object
buurt       object
dtype: object

In [32]:
#Cast column postcode to INT instead of OBJECT
postcode.postcode = postcode.postcode.astype('int64')
postcode.dtypes

postcode     int64
wijk        object
buurt       object
dtype: object

In [33]:
#Now merge with long/lat
postcode = pd.merge(postcode, neighlatlong, on='postcode')
postcode.head()

,postcode,wijk,buurt,id,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,10,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,11,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
2,1013,Haarlemmerbuurt,Westerdokseiland,12,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
3,1014,Westelijk Havengebied,Alfa-driehoek,13,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
4,1015,Grachtengordel-West,Langestraat e.o.,14,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres


In [34]:
#Now merge with demographic data
postcode = pd.merge(postcode, demo, on='postcode')
postcode.head()

,postcode,wijk,buurt,id,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,10,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,...,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,11,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,...,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38
2,1013,Haarlemmerbuurt,Westerdokseiland,12,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,...,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66
3,1014,Westelijk Havengebied,Alfa-driehoek,13,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,...,0.000000,0.000000,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74
4,1015,Grachtengordel-West,Langestraat e.o.,14,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,...,0.001346,0.000336,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51


In [35]:
#Drop unneeded columns
postcode.drop(['id', 'alternatieve_schrijfwijzen', 'netnummer', 'soort'], axis=1, inplace = True)
postcode.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9605.0,0.030713,...,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8245.0,0.026683,...,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21185.0,0.052396,...,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,580.0,0.051724,...,0.000000,0.000000,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,52.379093,4.885109,14860.0,0.038694,...,0.001346,0.000336,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51


In [36]:
#Now merge with income data
postcode = pd.merge(postcode, inkomen, on='postcode')
postcode.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,...,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9605.0,0.030713,...,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51,32.0,26.3
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8245.0,0.026683,...,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38,26.2,22.4
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21185.0,0.052396,...,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66,30.3,24.1
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,580.0,0.051724,...,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74,35.0,28.5
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,52.379093,4.885109,14860.0,0.038694,...,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51,33.3,27.2


In [37]:
postcode.shape

(248, 81)

In [38]:
postcode.dtypes

postcode                    int64
wijk                       object
buurt                      object
woonplaats                 object
gemeente                   object
provincie                  object
latitude                  float64
longitude                 float64
tot_inwoners              float64
tot_0-5                   float64
tot_5-10                  float64
tot_10-15                 float64
tot_15-20                 float64
tot_20-25                 float64
tot_25-30                 float64
tot_30-35                 float64
tot_35-40                 float64
tot_40-45                 float64
tot_45-50                 float64
tot_50-55                 float64
tot_55-60                 float64
tot_60-65                 float64
tot_65-70                 float64
tot_70-75                 float64
tot_75-80                 float64
tot_80-85                 float64
tot_85-90                 float64
tot_90-95                 float64
tot_95-                   float64
tot_man       

In [39]:
# Obtain latitude and longitude of the Netherlands
g = geocoder.arcgis('Alphen aan de Rijn, Netherlands') # Choice for Woerden as it centers on the Randstad area where our four selected cities are
NL_coords = g.latlng
NL_coords

[52.12743000000006, 4.658820000000048]

### Test map of all postcal codes selected on the map of the Netherlands

In [40]:
#Startpoint of map
nl_map = folium.Map(location=[NL_coords[0], NL_coords[1]], zoom_start=9)

#Add area markers for each postal code in file
for lat, lng, pc4 in zip(postcode['latitude'], postcode['longitude'], postcode['postcode']):
    label = pc4
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(nl_map)  

# display map
nl_map

In [41]:
# Obtain latitude and longitude of the Netherlands
g = geocoder.arcgis('Utrecht, Netherlands')
Utrecht_coords = g.latlng
print("Utrecht", Utrecht_coords)
g = geocoder.arcgis('Amsterdam, Netherlands')
amsterdam_coords = g.latlng
print("Amsterdam", amsterdam_coords)
g = geocoder.arcgis('Rotterdam, Netherlands')
rotterdam_coords = g.latlng
print("Rotterdam", rotterdam_coords)
g = geocoder.arcgis('Den Haag, Netherlands')
denhaag_coords = g.latlng
print("The Hague", denhaag_coords)

Utrecht [52.08965000000006, 5.114350000000059]
Amsterdam [52.36993000000007, 4.907880000000034]
Rotterdam [51.92282000000006, 4.478480000000047]
The Hague [52.08409000000006, 4.317320000000052]


### Obtain Foursquare data on venues near to postal code center point

Hidden cell below containing secret Foursquare credentials.

In [42]:
# The code was removed by Watson Studio for sharing.

In [43]:
# Funtion to explore neighbourhood in Foursquare

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['postcode', 
                  'latitude', 
                  'longitude', 
                  'venue', 
                  'venue latitude', 
                  'venue longitude', 
                  'venue category']
    
    return(nearby_venues)

In [44]:
venues = getNearbyVenues(names=postcode['postcode'],
                         latitudes=postcode['latitude'],
                         longitudes=postcode['longitude']
                         )

1011
1012
1013
1014
1015
1016
1017
1018
1019
1021
1022
1023
1024
1025
1026
1027
1028
1031
1032
1033
1034
1035
1036
1037
1041
1042
1043
1045
1046
1047
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1071
1072
1073
1074
1075
1076
1077
1078
1079
1081
1082
1083
1086
1087
1091
1092
1093
1094
1095
1096
1097
1098
1101
1102
1103
1104
1105
1106
1107
1108
1109
2491
2492
2493
2495
2496
2497
2498
2511
2512
2513
2514
2515
2516
2517
2518
2521
2522
2523
2524
2525
2526
2531
2532
2533
2541
2542
2543
2544
2545
2546
2547
2548
2551
2552
2553
2554
2555
2561
2562
2563
2564
2565
2566
2571
2572
2573
2574
2581
2582
2583
2584
2585
2586
2587
2591
2592
2593
2594
2595
2596
2597
3011
3012
3013
3014
3015
3016
3021
3022
3023
3024
3025
3026
3027
3028
3029
3031
3032
3033
3034
3035
3036
3037
3038
3039
3041
3042
3043
3044
3045
3046
3047
3051
3052
3053
3054
3055
3056
3059
3061
3062
3063
3064
3065
3066
3067
3068
3069
3071
3072
3073
3074
3075
3076
3077
3078
3079
3081
3082
3083


In [45]:
# Explore result
print(venues.shape)
venues.head()

(5259, 7)


,postcode,latitude,longitude,venue,venue latitude,venue longitude,venue category
0,1011,52.372976,4.903957,OCHA,52.374024,4.901683,Thai Restaurant
1,1011,52.372976,4.903957,Black Gold,52.371787,4.903056,Record Shop
2,1011,52.372976,4.903957,The Hendrick's Hotel,52.373597,4.906002,Hotel
3,1011,52.372976,4.903957,Eetcafé Bern,52.372575,4.900645,Swiss Restaurant
4,1011,52.372976,4.903957,HPS,52.371683,4.907673,Cocktail Bar


In [46]:
#Explore number per postal code
venues.groupby('postcode').count()

,latitude,longitude,venue,venue latitude,venue longitude,venue category
postcode,,,,,,
1011,100,100,100,100,100,100
1012,100,100,100,100,100,100
1013,7,7,7,7,7,7
1014,13,13,13,13,13,13
1015,100,100,100,100,100,100
1016,100,100,100,100,100,100
1017,100,100,100,100,100,100
1018,48,48,48,48,48,48
1019,28,28,28,28,28,28


In [47]:
#Explore unique categories
print('There are {} uniques categories.'.format(len(venues['venue category'].unique())))

There are 351 uniques categories.


In [48]:
# one hot encoding
venue_onehot = pd.get_dummies(venues[['venue category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
venue_onehot['postcode'] = venues['postcode']

#Explore
print(venue_onehot.shape)
venue_onehot.head()

(5259, 352)


,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit,postcode
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1011
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1011
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1011
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1011
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1011


In [49]:
#Group rows by neighborhood and take the mean of the frequency of occurrence of each category
venue_grouped = venue_onehot.groupby('postcode').mean().reset_index()
venue_grouped

,postcode,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,1011,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.010000,0.00,0.010000,...,0.00,0.0,0.01,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000
1,1012,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.010000,0.02,0.000000,...,0.00,0.0,0.00,0.01,0.000000,0.0,0.00,0.000000,0.000000,0.000000
2,1013,0.0,0.142857,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,...,0.00,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000
3,1014,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,...,0.00,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000
4,1015,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.01,0.000000,...,0.00,0.0,0.00,0.01,0.000000,0.0,0.01,0.020000,0.000000,0.000000
5,1016,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.030000,...,0.00,0.0,0.00,0.00,0.000000,0.0,0.00,0.040000,0.000000,0.000000
6,1017,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.01,0.010000,...,0.01,0.0,0.01,0.01,0.000000,0.0,0.01,0.010000,0.000000,0.000000
7,1018,0.0,0.000000,0.000000,0.0,0.0,0.020833,0.000000,0.00,0.000000,...,0.00,0.0,0.00,0.00,0.000000,0.0,0.00,0.020833,0.020833,0.291667
8,1019,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,...,0.00,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000
9,1021,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.066667,...,0.00,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000


In [50]:
#Explore size
venue_grouped.shape

(244, 352)

In [51]:
#Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
#Create new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venue_sorted = pd.DataFrame(columns=columns)
venue_sorted['postcode'] = venue_grouped['postcode']

for ind in np.arange(venue_grouped.shape[0]):
    venue_sorted.iloc[ind, 1:] = return_most_common_venues(venue_grouped.iloc[ind, :], num_top_venues)

venue_sorted

,postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1011,Bar,Hotel,Coffee Shop,Chinese Restaurant,Record Shop,Pub,Dessert Shop,Grocery Store,French Restaurant,Gay Bar
1,1012,Bar,Coffee Shop,Hotel,Marijuana Dispensary,French Restaurant,Café,Cocktail Bar,Dessert Shop,Burger Joint,Clothing Store
2,1013,Office,Theater,Dutch Restaurant,Advertising Agency,Board Shop,Restaurant,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market
3,1014,Music Venue,Soccer Field,Café,Hotel,Thrift / Vintage Store,Climbing Gym,Metro Station,Fast Food Restaurant,Nightclub,Farm
4,1015,Bar,Italian Restaurant,Café,Sandwich Place,Coffee Shop,Hotel,Marijuana Dispensary,Restaurant,Bistro,Spanish Restaurant
5,1016,Hotel,Bar,French Restaurant,Yoga Studio,Restaurant,Café,Art Gallery,Bookstore,Coffee Shop,Cheese Shop
6,1017,Italian Restaurant,Hotel,Marijuana Dispensary,Café,Clothing Store,Sushi Restaurant,Breakfast Spot,Burger Joint,Sandwich Place,Coffee Shop
7,1018,Zoo Exhibit,Restaurant,History Museum,Breakfast Spot,Museum,Seafood Restaurant,Science Museum,Plaza,Planetarium,Pizza Place
8,1019,Snack Place,Supermarket,Gastropub,Coffee Shop,Café,Furniture / Home Store,Tram Station,Farmers Market,Grocery Store,General Entertainment
9,1021,Brewery,Gymnastics Gym,Skate Park,Burger Joint,Rock Club,Seafood Restaurant,Mexican Restaurant,Supermarket,Hookah Bar,Turkish Restaurant


In [61]:
venue_sorted.shape

(244, 11)

### Cluster postal code areas based on venue type, demographic data and income data

Prepare dataset to cluster on

In [53]:
#Merge postcode income data with data on mean frequency of venue category
clustering = pd.merge(postcode,venue_grouped, on='postcode')

#Drop location names and geographic data
clustering = postcode.iloc[:, 8:]

#Normalize dataset
clustering = StandardScaler().fit_transform(clustering)

clustering

array([[ 0.1251347 , -1.17933201, -0.95526156, ..., -1.22549638,
         0.19135092,  0.51067319],
       [-0.12585654, -1.35680384, -1.60641964, ..., -1.60797463,
        -0.27365864,  0.05881782],
       [ 2.26225128, -0.22456853, -0.47622988, ..., -0.78417532,
         0.05505501,  0.25578042],
       ...,
       [-0.51710759,  0.05636636,  0.12231514, ..., -0.19574724,
         0.9449871 ,  0.88142632],
       [-0.31963654, -1.98094908, -1.59206259, ..., -2.01987429,
        -0.87496411, -0.85647897],
       [-1.61427149, -0.08542127,  1.54874682, ...,  0.8928447 ,
        -2.37421907, -2.53645408]])

Perform k-Means clustering

In [54]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1,
       2, 1, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 2,
       2, 2, 2, 1, 2, 1, 3, 3, 3, 3, 3, 1, 3, 3, 1, 1, 1, 2, 2, 3, 3, 3,
       3, 3, 3, 3, 2, 0, 2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 3,
       3, 3, 3, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       1, 2, 1, 1, 1, 1, 1, 3, 3, 3, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 3, 1, 1, 3, 3, 3, 2, 3, 3, 3, 2, 2, 3, 2, 2, 2,
       3, 2, 3, 3, 3, 2, 3, 2, 2, 3, 3, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 1,
       1, 2, 2, 3, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2,
       2, 3, 1, 1, 2, 1, 0, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 4, 3,
       3, 2, 2, 2, 1, 2, 2, 3, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 3, 1, 1, 3,
       2, 3, 3, 3, 3, 1], dtype=int32)

Create dataframe with all data including cluster number

In [56]:
kmeans.labels_.shape

(248,)

In [57]:
postcode.shape

(248, 81)

In [58]:
venue_sorted.shape

(244, 11)

In [59]:
postcode.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,...,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9605.0,0.030713,...,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51,32.0,26.3
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8245.0,0.026683,...,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38,26.2,22.4
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21185.0,0.052396,...,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66,30.3,24.1
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,580.0,0.051724,...,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74,35.0,28.5
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,52.379093,4.885109,14860.0,0.038694,...,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51,33.3,27.2


In [62]:
#Insert cluster number
postcode.insert(1, 'cluster', kmeans.labels_)

#Merge postal code dataframe with sorted venues dataframe
pcdata = pd.merge(postcode, venue_sorted, on='postcode')

pcdata.head()

,postcode,cluster,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1011,3,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9605.0,...,Bar,Hotel,Coffee Shop,Chinese Restaurant,Record Shop,Pub,Dessert Shop,Grocery Store,French Restaurant,Gay Bar
1,1012,3,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8245.0,...,Bar,Coffee Shop,Hotel,Marijuana Dispensary,French Restaurant,Café,Cocktail Bar,Dessert Shop,Burger Joint,Clothing Store
2,1013,3,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21185.0,...,Office,Theater,Dutch Restaurant,Advertising Agency,Board Shop,Restaurant,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market
3,1014,3,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,580.0,...,Music Venue,Soccer Field,Café,Hotel,Thrift / Vintage Store,Climbing Gym,Metro Station,Fast Food Restaurant,Nightclub,Farm
4,1015,3,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,52.379093,4.885109,14860.0,...,Bar,Italian Restaurant,Café,Sandwich Place,Coffee Shop,Hotel,Marijuana Dispensary,Restaurant,Bistro,Spanish Restaurant


### Show maps with postcode clusters

In [63]:
# create map
map_clusters = folium.Map(location=[NL_coords[0], NL_coords[1]], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pcdata['latitude'], pcdata['longitude'], pcdata['postcode'], pcdata['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

<b>Overall statistics as comparison</b>

In [85]:
pcdata.describe()

,postcode,cluster,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000
mean,2371.803279,2.016393,52.128215,4.673331,9068.463115,0.057986,0.050522,0.045956,0.049911,0.085078,0.098287,0.089432,0.075509,0.071411,0.069742,0.064474,0.059297,0.051426,0.046577,0.028881,0.022707,0.015710,0.010429,0.004886,0.001361,0.503504,0.029738,0.025511,0.023580,0.024617,0.039229,0.049287,0.046890,0.038752,0.040039,0.035912,0.034427,0.030362,0.024662,0.023545,0.013705,0.010262,0.006718,0.003335,0.001156,0.000431,0.499288,0.027963,0.024797,0.024130,0.024837,0.044432,0.051315,0.043518,0.035076,0.034500,0.034312,0.031266,0.027572,0.025891,0.022845,0.014742,0.012426,0.009098,0.006641,0.003702,0.001122,0.463788,0.154224,0.312550,4748.155738,0.499532,0.216768,0.279174,1.926025,29.693033,21.982787
std,956.495516,0.893356,0.176014,0.306128,5358.851337,0.022510,0.021389,0.020696,0.023884,0.053233,0.047210,0.038780,0.035111,0.033078,0.021390,0.019450,0.024005,0.028732,0.023553,0.013814,0.015575,0.011537,0.011751,0.005742,0.002195,0.053701,0.012784,0.011501,0.011111,0.011261,0.024402,0.034841,0.031990,0.017021,0.034797,0.013256,0.014906,0.016220,0.011938,0.015803,0.007445,0.005995,0.006102,0.003396,0.001489,0.003061,0.059172,0.012322,0.012143,0.032274,0.014043,0.032366,0.028547,0.021630,0.011096,0.015419,0.032809,0.013914,0.010725,0.024698,0.013798,0.007848,0.008184,0.007401,0.006818,0.004523,0.001983,0.190038,0.076569,0.194420,2875.311055,0.144019,0.057252,0.119542,0.337833,12.172474,8.428427
min,1011.000000,0.000000,51.871742,4.218229,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.445011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,1.130000,0.000000,0.000000
25%,1090.000000,1.000000,51.962515,4.410400,6083.750000,0.047979,0.039184,0.034128,0.039050,0.055999,0.065315,0.068219,0.064759,0.062149,0.060575,0.056466,0.048587,0.040

Some key stats:
- Average inhabitants: 9068.463115
- Average percentage females: 49.9288%
- Average with migration background non-western:  31.2550%
- Average single person households: 49.9532%
- Average standardised income: 21.982787

<b>Cluster 1</b>

In [73]:
pcdata.loc[pcdata['cluster'] == 0]

,postcode,cluster,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,1041,0,Westelijk Havengebied,Petroleumhaven,Amsterdam,Amsterdam,Noord-Holland,52.414315,4.839814,25.0,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.600000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.0000,0.000000,0.0000,0.0,0.0,0.000000,0.200000,0.0,0.0,0.0,0.0,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.400000,0.000000,0.200000,10.0,1.000000,0.500000,0.000000,1.50,0.0,0.0,Harbor / Marina,Canal,Zoo Exhibit,Frozen Yogurt Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
24,1042,0,Westelijk Havengebied,Vervoerscentrum,Amsterdam,Amsterdam,Noord-Holland,52.404253,4.824768,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333,10.0,1.000000,0.000000,0.000000,1.45,0.0,0.0,Harbor / Marina,Zoo Exhibit,Fruit & Vegetable Store,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
25,1043,0,Westelijk Havengebied,Westhaven Noord,Amsterdam,Amsterdam,Noord-Holland,52.388623,4.815402,50.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.900000,0.000000,0.000000,0.0,0.000000,0.100000,0.300000,0.200000,0.100000,0.200000,0.000000,0.100000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.0,0.0,0.000000,0.100000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.700000,0.500000,0.300000,45.0,0.777778,0.111111,0.000000,1.19,0.0,0.0,Music Venue,Garden,Tunnel,Harbor / Marina,French Restaurant,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Film Studio
26,1046,0,Westelijk Havengebied,Amerikahaven,Amsterdam,Amsterdam,Noord-Holland,52.389960,4.782460,35.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.142857,0.142857,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.571429,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,0.142857,0.142857,0.142857,0.000000,0.000000,0.142857,0.000000,0.142857,0.0000,0.000000,0.0000,0.0,0.0,0.000000,0.428571,0.0,0.0,0.0,0.0,0.000000,0.142857,0.000000,0.000000,0.

In [78]:
pcdata.loc[pcdata['cluster'] == 0].describe()

,postcode,cluster,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
count,10.000000,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.0,10.000000,10.0,10.0,10.000000,10.000000,10.000000,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.0,10.0,10.000000,10.000000,10.0,10.0,10.0,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.0,10.0,10.0,10.0,10.0,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.0,10.0
mean,1854.000000,0.0,52.199457,4.665291,63.500000,0.031382,0.009954,0.003704,0.009954,0.091594,0.147943,0.159372,0.066938,0.048624,0.081587,0.052493,0.102348,0.105516,0.049894,0.014716,0.016667,0.0,0.004762,0.0,0.0,0.684339,0.003704,0.003704,0.0,0.004762,0.032222,0.113499,0.138307,0.062017,0.077837,0.038790,0.041382,0.052705,0.023181,0.039418,0.006250,0.004762,0.006250,0.0,0.0,0.004762,0.325661,0.0,0.0,0.0,0.0,0.018419,0.093075,0.047407,0.008466,0.048419,0.020370,0.027778,0.011012,0.061799,0.021429,0.0,0.0,0.0,0.0,0.0,0.0,0.450800,0.234557,0.220694,35.500000,0.598779,0.200413,0.113939,1.574000,0.0,0.0
std,1033.883187,0.0,0.243322,0.212654,46.729125,0.053203,0.021826,0.011712,0.021826,0.077718,0.107164,0.101578,0.071228,0.054578,0.076766,0.057844,0.081648,0.107018,0.076974,0.024450,0.052705,0.0,0.015058,0.0,0.0,0.166842,0.011712,0.011712,0.0,0.015058,0.051971,0.129733,0.103705,0.062008,0.072598,0.053504,0.055975,0.067944,0.038137,0.063498,0.019764,0.015058,0.019764,0.0,0.0,0.015058,0.144592,0.0,0.0,0.0,0.0,0.024776,0.081701,0.083674,0.018021,0.062781,0.052705,0.054691,0.023479,0.113803,0.053181,0.0,0.0,0.0,0.0,0.0,0.0,0.270485,0.208979,0.143377,32.356521,0.326014,0.155901,0.123530,0.288452,0.0,0.0
min,1041.000000,0.0,51.871742,4.400721,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.100000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.062500,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,1.140000,0.0,0.0
25%,1043.750000,0.0,51.935008,4.440799,26.250000,0.000000,0.000000,0.000000,0.000000,0.011905,0.074653,0.082589,0.000000,0.000000,0.000000,0.000000,0.037037,0.009259,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.578571,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.063492,0.000000,0.009259,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.205556,0.0,0.0,0.0,0.0,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.233333,0.065394,0.096875,12.500000,0.397727,0.092995,0.000000,1.462500,0.0,0.0
50%,1074.000000,0.0,52.34870

Key stats:
- Average inhabitants: 63.500000
- Average percentage females: 32.5661%
- Average with migration background non-western:  22.0694%
- Average single person households: 59.8779%
- Average standardised income: 0.0

Analysis:
- Almost no inhabitants, almost no income.
- Venues include mostly harbor/marina, snack places and leasure zones
- See also map and area names: industrial and harbour area

Conclusion:
- Cluster is harbour area. Not suited for settling our salad bar

<b>Cluster 2</b>

In [74]:
pcdata.loc[pcdata['cluster'] == 1]

,postcode,cluster,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,1023,1,Noordelijke IJ-oevers Oost,Bedrijventerrein Nieuwendammerdijk,Amsterdam,Amsterdam,Noord-Holland,52.387080,4.949686,4630.0,0.052916,0.053996,0.049676,0.055076,0.034557,0.041037,0.050756,0.060475,0.065875,0.082073,0.091793,0.077754,0.063715,0.074514,0.050756,0.043197,0.030238,0.014039,0.005400,0.001080,0.471922,0.028078,0.026998,0.025918,0.026998,0.016199,0.021598,0.021598,0.028078,0.026998,0.037797,0.043197,0.037797,0.031317,0.036717,0.022678,0.020518,0.015119,0.005400,0.001080,0.000000,0.528078,0.025918,0.026998,0.023758,0.028078,0.018359,0.020518,0.029158,0.033477,0.038877,0.043197,0.048596,0.041037,0.032397,0.037797,0.028078,0.021598,0.016199,0.008639,0.004320,0.001080,0.226782,0.117711,0.109071,2330.0,0.424893,0.255365,0.319742,1.98,32.9,23.7,Harbor / Marina,Bus Stop,Bakery,Pharmacy,Park,Smoke Shop,Plaza,Food Service,Forest,Farm
13,1025,1,IJplein/Vogelbuurt,Vliegenbos,Amsterdam,Amsterdam,Noord-Holland,52.399622,4.936959,15025.0,0.056240,0.052246,0.055574,0.057238,0.072879,0.073544,0.066223,0.053245,0.057571,0.066889,0.067887,0.063228,0.053245,0.054576,0.043927,0.040266,0.031281,0.021963,0.009651,0.002995,0.480865,0.028286,0.027288,0.029950,0.028286,0.036938,0.035940,0.032945,0.026622,0.027953,0.032612,0.032280,0.031947,0.026290,0.025957,0.019967,0.016306,0.011647,0.006656,0.002662,0.000666,0.519468,0.027953,0.024958,0.025624,0.028619,0.035940,0.037271,0.032945,0.026622,0.029950,0.034609,0.035607,0.031614,0.027288,0.028952,0.023960,0.023627,0.019634,0.015308,0.006988,0.002329,0.500166,0.124459,0.375707,7520.0,0.492021,0.218085,0.289894,1.95,28.5,21.3,Supermarket,Department Store,Clothing Store,Diner,Electronics Store,Grocery Store,Martial Arts Dojo,Market,Gift Shop,Gym / Fitness Center
14,1026,1,Waterland,Zunderdorp,Amsterdam,Amsterdam,Noord-Holland,52.381232,4.993553,515.0,0.048544,0.077670,0.077670,0.058252,0.029126,0.019417,0.038835,0.048544,0.087379,0.097087,0.097087,0.087379,0.058252,0.058252,0.058252,0.029126,0.009709,0.019417,0.009709,0.000000,0.543689,0.029126,0.048544,0.038835,0.038835,0.009709,0.009709,0.019417,0.019417,0.038835,0.038835,0.048544,0.048544,0.038835,0.038835,0.029126,0.019417,0.009709,0.009709,0.000000,0.000000,0.466019,0.019417,0.038835,0.029126,0.019417,0.009709,0.009709,0.019417,0.029126,0.048544,0.048544,0.048544,0.029126,0.019417,0.029126,0.029126,0.019417,0.000000,0.009709,0.000000,0.000000,0.116505,0.097087,0.019417,210.0,0.261905,0.333333,0.404762,2.45,52.0,33.8,Harbor / Marina,Boarding House,Soccer Field,Food,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
15,1027,1,Waterland,Zunderdorp,Amsterdam,Amsterdam,Noord-Holland,52.411901,4.962458,450.0,0.033333,0.044444,0.055556,0.066667,0.066667,0.033333,0.022222,0.044444,0.055556,0.088889,0.100000,0.077778,0.077778,0.100000,0.066667,0.055556,0.022222,0.011111,0.011111,0.000000,0.533333,0.011111,0

In [79]:
pcdata.loc[pcdata['cluster'] == 1].describe()

,postcode,cluster,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
count,64.000000,64.0,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000
mean,2431.265625,1.0,52.106000,4.583926,7884.453125,0.050728,0.052353,0.051149,0.051531,0.056957,0.059767,0.058280,0.058132,0.065482,0.071894,0.073860,0.070198,0.065083,0.067106,0.044874,0.037289,0.030418,0.020116,0.010680,0.003120,0.486731,0.026126,0.026008,0.026721,0.026306,0.028698,0.029545,0.028491,0.028386,0.031688,0.035666,0.037661,0.035429,0.032864,0.032590,0.020703,0.016145,0.012251,0.006867,0.002607,0.000548,0.514327,0.024644,0.024549,0.024460,0.024197,0.027280,0.029566,0.030781,0.030878,0.033775,0.035391,0.036916,0.035866,0.032072,0.033677,0.023961,0.022791,0.017750,0.013764,0.008102,0.002619,0.340978,0.147762,0.194456,3993.828125,0.452300,0.254661,0.296911,1.995938,34.639063,25.329687
std,854.873002,0.0,0.170598,0.299901,5288.295110,0.014897,0.012604,0.013719,0.012639,0.025814,0.021668,0.019944,0.014596,0.011813,0.014677,0.016142,0.014207,0.016762,0.016771,0.011528,0.011716,0.011143,0.012256,0.007903,0.003455,0.026069,0.008336,0.008256,0.007063,0.007964,0.015779,0.012324,0.010488,0.008721,0.005050,0.009038,0.011994,0.009337,0.011948,0.010873,0.007004,0.005730,0.005768,0.004632,0.002101,0.000603,0.024128,0.007856,0.007655,0.008075,0.008193,0.014062,0.012502,0.009588,0.007810,0.008508,0.008888,0.009623,0.008897,0.009199,0.008548,0.007107,0.007773,0.008391,0.008997,0.006403,0.003238,0.136557,0.070789,0.132477,2685.331073,0.094961,0.052286,0.067483,0.229781,15.167913,10.461504
min,1023.000000,1.0,51.873795,4.218229,130.000000,0.000000,0.026144,0.024357,0.025822,0.000000,0.017188,0.000000,0.000000,0.027778,0.049391,0.043302,0.045332,0.036723,0.037429,0.023256,0.000000,0.009709,0.000000,0.000000,0.000000,0.445011,0.000000,0.000000,0.013532,0.000000,0.000000,0.000000,0.000000,0.000000,0.019621,0.023256,0.020298,0.021893,0.017614,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.437500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.020725,0.000000,0.015625,0.000000,0.012439,0.000000,0.000000,0.000000,0.000000,0.069767,0.046512,0.019417,65.000000,0.230769,0.185419,0.140558,1.500000,0.000000,0.000000
25%,2145.500000,1.0,51.958935,4.311830,4223.750000,0.045883,0.044690,0.041883,0.042632,0.043069,0.045863,0.047964,0.051293,0.058127,0.061568,0.061912,0.061640,0.055393,0.055311,0.037911,0.030300,0.023420,0.013067,0.006173,0.000908,0.471282,0.023428,0.022575,0.022

Key stats:
- Average inhabitants: 7884.453125
- Average percentage females: 51,4327%
- Average with migration background non-western:  19,4456%
- Average single person households: 45,2300%
- Average standardised income: 25.329687
    
Analysis:
- High population density
- Lower than average non-western migration
- Higher than average income
- Venues mostly leasure, sport and shopping

Conclusion:
- Cluster is residential zone, few restaurants. Not ideal for a salad bar, although it has a high average income and relatively many females.

<b>Cluster 3</b>

In [75]:
pcdata.loc[pcdata['cluster'] == 2]

,postcode,cluster,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,1019,2,Oostelijke Eilanden/Kadijken,Marine-Etablissement,Amsterdam,Amsterdam,Noord-Holland,52.372858,4.938828,18530.0,0.059093,0.060712,0.056125,0.054236,0.041015,0.062601,0.078521,0.080140,0.089854,0.101457,0.104155,0.083378,0.053427,0.038586,0.019428,0.009984,0.004857,0.001889,0.000540,0.000270,0.504317,0.029951,0.031301,0.027523,0.026174,0.019968,0.030221,0.039665,0.039665,0.043443,0.050998,0.054506,0.044792,0.027793,0.019698,0.009714,0.004587,0.002159,0.000809,0.000270,0.000000,0.495953,0.029412,0.029682,0.028332,0.028063,0.021047,0.032110,0.038856,0.040475,0.046411,0.050189,0.049919,0.038316,0.025634,0.018618,0.009984,0.005397,0.002698,0.000809,0.000270,0.000000,0.390178,0.180248,0.209930,9125.0,0.427397,0.249863,0.322740,2.02,42.6,31.5,Snack Place,Supermarket,Gastropub,Coffee Shop,Café,Furniture / Home Store,Tram Station,Farmers Market,Grocery Store,General Entertainment
9,1021,2,IJplein/Vogelbuurt,Vogelbuurt Zuid,Amsterdam,Amsterdam,Noord-Holland,52.384278,4.920920,6980.0,0.060888,0.050143,0.049427,0.046562,0.077364,0.094556,0.095989,0.075215,0.075931,0.073782,0.074499,0.065903,0.053009,0.041547,0.026504,0.021490,0.011461,0.005014,0.001433,0.000000,0.508596,0.034384,0.027221,0.024355,0.023639,0.035817,0.046562,0.046562,0.039398,0.040831,0.039398,0.037966,0.034384,0.027221,0.020057,0.012894,0.010745,0.005731,0.001433,0.000716,0.000000,0.491404,0.026504,0.023639,0.025072,0.022206,0.040831,0.047994,0.049427,0.035817,0.034384,0.034384,0.036533,0.031519,0.026504,0.021490,0.012894,0.011461,0.006447,0.003582,0.000716,0.000000,0.603152,0.133238,0.469914,3745.0,0.543391,0.190921,0.264352,1.86,24.4,18.9,Brewery,Gymnastics Gym,Skate Park,Burger Joint,Rock Club,Seafood Restaurant,Mexican Restaurant,Supermarket,Hookah Bar,Turkish Restaurant
10,1022,2,IJplein/Vogelbuurt,Vliegenbos,Amsterdam,Amsterdam,Noord-Holland,52.387908,4.929879,2745.0,0.105647,0.074681,0.054645,0.043716,0.041894,0.089253,0.120219,0.116576,0.094718,0.078324,0.054645,0.040073,0.025501,0.027322,0.018215,0.009107,0.005464,0.001821,0.001821,0.000000,0.511840,0.052823,0.036430,0.029144,0.025501,0.020036,0.041894,0.058288,0.056466,0.052823,0.041894,0.029144,0.018215,0.014572,0.014572,0.009107,0.001821,0.003643,0.001821,0.000000,0.000000,0.488160,0.052823,0.038251,0.025501,0.018215,0.021858,0.047359,0.061931,0.056466,0.041894,0.036430,0.023679,0.020036,0.010929,0.012750,0.007286,0.007286,0.001821,0.000000,0.001821,0.000000,0.457195,0.143898,0.311475,1245.0,0.397590,0.200803,0.401606,2.20,33.3,23.4,Park,Bus Stop,Rock Club,Music Venue,Café,Camera Store,Campground,Skate Park,Boat or Ferry,Caribbean Restaurant
12,1024,2,Waterlandpleinbuurt,Baanakkerspark Zuid,Amsterdam,Amsterdam,Noord-Holland,52.393495,4.955826,13300.0,0.078947,0.071053,0.069925,0.072180,0.062030,0.078947,0.088346,0.072556,0.065789,0.064662,0.064662,0.050752,0.036842,0.037218,0.025940,0.024436,0.018421,0.

In [80]:
pcdata.loc[pcdata['cluster'] == 2].describe()

,postcode,cluster,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
count,83.000000,83.0,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,2410.204819,2.0,52.105703,4.627404,10401.506024,0.074242,0.068439,0.062109,0.059810,0.072192,0.083469,0.084252,0.078406,0.076876,0.074535,0.067320,0.055267,0.043185,0.036614,0.022943,0.017768,0.011509,0.006992,0.003237,0.000845,0.499965,0.038276,0.034964,0.032068,0.030206,0.035540,0.040411,0.041191,0.038993,0.038913,0.038798,0.034391,0.028497,0.021512,0.018448,0.011228,0.008349,0.004743,0.002321,0.000735,0.000132,0.500095,0.036064,0.033596,0.029960,0.029671,0.036576,0.042977,0.043105,0.039342,0.037865,0.035696,0.032846,0.026721,0.021701,0.018216,0.011724,0.009349,0.006738,0.004697,0.002400,0.000679,0.583123,0.123069,0.459970,4800.120482,0.418285,0.200786,0.380858,2.194458,31.019277,21.854217
std,897.550919,0.0,0.171962,0.298188,5230.742598,0.018195,0.016739,0.015756,0.011742,0.024088,0.025645,0.020942,0.014318,0.016461,0.015939,0.013703,0.014091,0.012606,0.009772,0.006712,0.006330,0.005838,0.004961,0.002862,0.001096,0.011715,0.009599,0.008789,0.008602,0.006168,0.011527,0.012849,0.011084,0.007622,0.007879,0.007872,0.006708,0.006780,0.006223,0.004958,0.003147,0.002914,0.002233,0.001520,0.000738,0.000241,0.011657,0.009039,0.008265,0.007558,0.005909,0.013029,0.013064,0.010205,0.007894,0.009484,0.008802,0.007731,0.007883,0.006699,0.005105,0.003689,0.003735,0.003793,0.003679,0.002253,0.000829,0.194611,0.040540,0.197258,2472.743117,0.110523,0.036252,0.100741,0.306655,8.630017,4.869600
min,1019.000000,2.0,51.874654,4.261203,1355.000000,0.049096,0.044053,0.041929,0.033543,0.018868,0.018581,0.029561,0.053824,0.057206,0.035088,0.024123,0.015351,0.006579,0.008772,0.004386,0.000000,0.000000,0.000000,0.000000,0.000000,0.463942,0.024548,0.021964,0.017544,0.015351,0.006579,0.009291,0.009291,0.022663,0.026423,0.021930,0.013158,0.008772,0.002193,0.005698,0.002193,0.001220,0.000000,0.000000,0.000000,0.000000,0.474037,0.023602,0.022026,0.016771,0.017724,0.008386,0.010135,0.020270,0.029745,0.024254,0.013158,0.013158,0.006579,0.002193,0.004386,0.002439,0.000000,0.000000,0.000000,0.000000,0.000000,0.196429,0.068199,0.074603,565.000000,0.126649,0.123864,0.254083,1.790000,21.400000,15.900000
25%,1105.000000,2.0,51.950578,4.367265,7012.500000,0.063720,0.059003,0.051507,0.052316,0.057825,0.070681,0.073532,0.069390,0.065910,0.065384,0.060315,0.047608,0.036871,0.032410,0.019206,0.013864,0.007476,0.003699,0.001407,0.000000,0.493263,0.032366,0.029965,0.

Key stats:
- Average inhabitants: 10401.506024
- Average percentage females: 50.0095%
- Average with migration background non-western: 45,9970%
- Average single person households: 41,8285%
- Average standardised income: 21.854217

Analysis:
- Very high population density
- Relatively high migration background, higher than average income, but lower than cluster 2.
- Venues inludes many restaurants
- See also map, city centers

Conclusion:
- Cluster is densily populated city center area with many restaurants.  Very good candidate for opening salad bar.

Cluster 4

In [76]:
pcdata.loc[pcdata['cluster'] == 3]

,postcode,cluster,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1011,3,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9605.0,0.030713,0.030193,0.022905,0.032795,0.092140,0.114524,0.086934,0.067673,0.064029,0.067673,0.080167,0.081728,0.078084,0.069755,0.037480,0.020822,0.011973,0.006247,0.003644,0.001562,0.519521,0.015096,0.014055,0.010932,0.015617,0.038522,0.053618,0.045809,0.039563,0.036960,0.040083,0.047892,0.044768,0.042686,0.036439,0.019781,0.010411,0.005206,0.001562,0.001041,0.000521,0.480479,0.015617,0.015617,0.011973,0.016658,0.053618,0.060906,0.041124,0.028110,0.027069,0.027590,0.032275,0.036439,0.035919,0.033316,0.017699,0.010411,0.006767,0.004685,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51,32.0,26.3,Bar,Hotel,Coffee Shop,Chinese Restaurant,Record Shop,Pub,Dessert Shop,Grocery Store,French Restaurant,Gay Bar
1,1012,3,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8245.0,0.026683,0.016374,0.013948,0.019406,0.133414,0.206792,0.141298,0.084900,0.066101,0.061856,0.059430,0.050940,0.040631,0.036386,0.021831,0.010309,0.004851,0.001819,0.000606,0.000000,0.545179,0.012735,0.007277,0.007277,0.007884,0.055791,0.102486,0.078229,0.053366,0.043663,0.038205,0.040631,0.032747,0.023651,0.020619,0.012735,0.006671,0.002426,0.000606,0.000606,0.000000,0.454215,0.014554,0.009096,0.007884,0.012129,0.078229,0.104912,0.063675,0.031534,0.022438,0.022438,0.019406,0.018193,0.016980,0.016374,0.009703,0.004851,0.002426,0.001213,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38,26.2,22.4,Bar,Coffee Shop,Hotel,Marijuana Dispensary,French Restaurant,Café,Cocktail Bar,Dessert Shop,Burger Joint,Clothing Store
2,1013,3,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21185.0,0.052396,0.040359,0.031862,0.032570,0.060892,0.097947,0.101251,0.090158,0.078829,0.079537,0.076941,0.069389,0.063016,0.053576,0.031626,0.018881,0.012037,0.005664,0.002360,0.000708,0.501770,0.025490,0.021005,0.016285,0.016757,0.027614,0.044843,0.050979,0.043899,0.042011,0.043191,0.041539,0.034930,0.031626,0.027142,0.016521,0.009913,0.005664,0.001888,0.000708,0.000000,0.498230,0.026906,0.019353,0.015577,0.015813,0.033278,0.053104,0.050271,0.046259,0.036819,0.036346,0.035166,0.034458,0.031390,0.026434,0.015105,0.008969,0.006608,0.003776,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66,30.3,24.1,Office,Theater,Dutch Restaurant,Advertising Agency,Board Shop,Restaurant,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market
3,1014,3,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,580.0,0.051724,0.025862,0.008621,0.025862,0.043103,0.232759,0.215517,0.103448,0.086207,0.060345,0.043103,0.043103,0.025862,0.017241,0.017241,0.017241,0.008621,0.000000,0.000000,0.000000,0.577

In [81]:
pcdata.loc[pcdata['cluster'] == 3].describe()

,postcode,cluster,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
count,86.000000,86.0,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,2337.255814,3.0,52.158884,4.780368,9815.465116,0.051467,0.037172,0.031950,0.044379,0.118673,0.136624,0.110522,0.081706,0.068215,0.062949,0.056884,0.050758,0.043524,0.041070,0.024694,0.017588,0.010829,0.007319,0.002790,0.000724,0.498415,0.027560,0.018850,0.016067,0.020560,0.051898,0.065652,0.055999,0.043978,0.037599,0.033393,0.031648,0.026148,0.022057,0.020161,0.011915,0.008490,0.004640,0.002111,0.000629,0.000134,0.501683,0.026193,0.019662,0.015529,0.023826,0.068320,0.071289,0.053449,0.037586,0.030573,0.028380,0.026304,0.024466,0.021461,0.019680,0.012680,0.009272,0.006101,0.004067,0.002157,0.000580,0.441098,0.175741,0.266659,5862.441860,0.607361,0.208415,0.184481,1.656628,28.530233,22.427907
std,1063.442116,0.0,0.172467,0.298392,4595.002889,0.014197,0.012225,0.011072,0.031355,0.066472,0.030838,0.024749,0.015114,0.012639,0.011532,0.012434,0.012721,0.011855,0.013648,0.007880,0.006177,0.004411,0.011775,0.002145,0.000724,0.023735,0.011678,0.006528,0.006142,0.012328,0.028646,0.016166,0.014395,0.010901,0.010390,0.006944,0.010867,0.006638,0.005967,0.006398,0.004042,0.002931,0.001953,0.001219,0.000573,0.000226,0.023678,0.011523,0.011414,0.005803,0.019400,0.040714,0.016696,0.013468,0.008780,0.007622,0.006107,0.006790,0.006956,0.006419,0.006302,0.004492,0.003615,0.002985,0.002354,0.001666,0.000686,0.139004,0.056775,0.141705,2797.752643,0.074771,0.038956,0.053669,0.159747,6.754068,5.282550
min,1011.000000,3.0,51.887200,4.268899,45.000000,0.000000,0.000000,0.000000,0.000000,0.043103,0.033333,0.030577,0.019458,0.020153,0.025017,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.460884,0.000000,0.000000,0.000000,0.000000,0.022031,0.016667,0.000000,0.010424,0.009729,0.000000,0.011119,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.422414,0.000000,0.000000,0.000000,0.000000,0.025862,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.177143,0.000000,0.051429,35.000000,0.457627,0.055556,0.049011,1.130000,0.000000,0.000000
25%,1075.250000,3.0,52.070407,4.474178,7208.750000,0.045615,0.030274,0.024142,0.032233,0.084447,0.114676,0.098290,0.072359,0.062555,0.057345,0.051336,0.044628,0.036419,0.031760,0.019481,0.014397,0.008196,0.004013,0.001268,0.000000,0.483611,0.023399,0.015313,0.01255

Cluster 5

In [77]:
pcdata.loc[pcdata['cluster'] == 4]

,postcode,cluster,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
215,3528,4,Wijk 09 Leidsche Rijn,Bedrijvengebied Papendorp,Utrecht,Utrecht,Utrecht,52.068572,5.082438,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,10.0,0.0,0.0,0.5,1.86,0.0,0.0,Hotel,Cafeteria,Brasserie,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant


In [82]:
pcdata.loc[pcdata['cluster'] == 4].describe()

,postcode,cluster,latitude,longitude,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,tot_40-45,tot_45-50,tot_50-55,tot_55-60,tot_60-65,tot_65-70,tot_70-75,tot_75-80,tot_80-85,tot_85-90,tot_90-95,tot_95-,tot_man,man_0-5,man_5-10,man_10-15,man_15-20,man_20-25,man_25-30,man_30-35,man_35-40,man_40-45,man_45-50,man_50-55,man_55-60,man_60-65,man_65-70,man_70-75,man_75-80,man_80-85,man_85-90,man_90-95,man_95-,tot_vrouw,vrouw_0-5,vrouw_5-10,vrouw_10-15,vrouw_15-20,vrouw_20-25,vrouw_25-30,vrouw_30-35,vrouw_35-40,vrouw_40-45,vrouw_45-50,vrouw_50-55,vrouw_55-60,vrouw_60-65,vrouw_65-70,vrouw_70-75,vrouw_75-80,vrouw_80-85,vrouw_85-90,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
count,1.0,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,1.0
mean,3528.0,4.0,52.068572,5.082438,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,10.0,0.0,0.0,0.5,1.86,0.0,0.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,3528.0,4.0,52.068572,5.082438,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,10.0,0.0,0.0,0.5,1.86,0.0,0.0
25%,3528.0,4.0,52.068572,5.082438,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,10.0,0.0,0.0,0.5,1.86,0.0,0.0
50%,3528.0,4.0,52.068572,5.082438,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,10.0,0.0,0.0,0.5,1.86,0.0,0.0
75%,3528.0,4.0,52.068572,5.082438,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,10.0,0.0,0.0,0.5,1.86,0.0,0.0
max,3528.0,4.0,52.068572,5.082438,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,10.0,0.0,0.0,0.5,1.86,0.0,0.0


Analysis:
- Very similar to cluster 1, but no harbours.
- Also almost no inhabitants, almost no income.
- Venues include mostly snack places and leasure zones
See also map and area names: industrial area

Conclusion:

Cluster is industrial area. Not suited for settling our salad bar